In [1]:
from urllib.request import urlopen
import urllib
from bs4 import BeautifulSoup
import csv
import os
import requests
import re
import pymysql

In [2]:
def getClassification():################获取全部图书分类信息
        resp=urlopen('http://category.dangdang.com/?ref=www-0-C')
        soup=BeautifulSoup(resp,'html.parser')
        ##############目录
        General_classification =soup.find('div',id='floor_1')
    
        Gclass,Sclass,sclassion=[],[],[]###总目录和分目录获取
        for i in General_classification:
            try:
                Gclass.append(i.a.string.encode('iso-8859-1').decode('gb2312'))####需要编码解码消除乱码
                Sclass.append(i.ul)
            except:
                Gclass.append('港台图书')###特殊处理
                Sclass.append(i.ul)  
                
        del Gclass[0];del Gclass[-1];del Sclass[0];del Gclass[-1] #####删除头尾无用部分  53-->51个数
        
        sclassion,sclassurl,lensclass=[],[],[] #####分目录名，url,每个分类的长度，调用时sclassion[x:x+1]
        for i in range(len(Gclass)):
            lensc=0    ###计数每个分类的个数
            for j in Sclass[i]:
                if j.a.string.encode('iso-8859-1').decode('gb2312')=='更多':
                     continue
                sclassion.append(j.a.string.encode('iso-8859-1').decode('gb2312'))
                sclassurl.append(j.a.get('href'))
                lensc+=1
            lensclass.append(lensc)        
            
        return Gclass,sclassion,sclassurl,lensclass

pivot=0
for i in range(len(Gclass)):
    #print(Gclass[i],sclassion[lensclass[i]:lensclass[i+1],sclassion[lensclass[i]:lensclass[i+1])
    if Gclass[i]=='韩文原版书':
        continue
    print(Gclass[i])
    print(sclassion[pivot:pivot+lensclass[i]])
    print(sclassurl[pivot:pivot+lensclass[i]])
    pivot+=lensclass[i]

In [3]:
def getbookinfo(Gclass,sclassion,sclassurl,lensclass):################获取图书信息
        pivot=0###########标定位置，方便小分类分段截取
        for i in range(len(Gclass)):
            if Gclass[i]=='韩文原版书':#####去掉空
                continue
            gclass=Gclass[i]            ####大分类49
            sclass=sclassion[pivot:pivot+lensclass[i]]######小分类679
            bookurl=sclassurl[pivot:pivot+lensclass[i]]   #######小分类url-679
            pivot+=lensclass[i] 
            
            #############  1----15-----15
            for _ in range(lensclass[i]):
                url=bookurl[_]
                html = requests.get(url)
                html.encoding = 'gb2312'
                soup = BeautifulSoup(html.text, 'html.parser')########对url进行处理 ，进入分类书籍页面
                book_title=soup.find_all('a',dd_name='单品图片')  #####为获取url集准备
                for i in book_title:
                    try:
                        resp=urlopen(i.get('href'))
                        soup=BeautifulSoup(resp,'html.parser')
                        booktitle=soup.find('h1').get('title')    #####进入具体某本书的页面
                        booktitle=booktitle[0:10]
                        bookdetail=soup.find('span',class_='head_title_name').get('title')
                        bookdetail=bookdetail[0:19]
                        bookprice=float(soup.find('p',id='dd-price').text[2:].strip())
                        author=soup.find('a',dd_name='作者').string
                        press=soup.find('a',dd_name='出版社').string
                        publicationtime=soup.find_all('span',class_='t1')[2].text
                        publicationtime=publicationtime[5:-1]
                        bookinfo=[booktitle,publicationtime, author, press,bookprice,bookdetail]
                        writemysql(bookinfo,gclass,sclass[_])
                    except:
                        pass
            

def writeData(bookinfo,Gclass,sclassion):
    
    Gclass,sclassion=Gclass.replace('/','-'),sclassion.replace('/','-')#####消除文件名中'/'
    
    path = 'dangdang-book/'+ Gclass+'/'+sclassion # 设置路径，也可设为相对路径

    # 判断目录是否存在
    if not os.path.exists(path):
        # 目录不存在创建，makedirs可以创建多级目录
        os.makedirs(path)
    try:
        # 保存数据到文件
        with open(path+sclassion+'.csv','a',encoding='utf-8-sig',newline='') as f:
            writer=csv.writer(f,dialect='excel')
            writer.writerow(bookinfo)
        #print('保存成功')
    except Exception as e:
        print('保存失败', e)

In [4]:
def writemysql(bookinfo,Gclass,sclassion):
    Gclass,sclassion=Gclass.replace('/','-'),sclassion.replace('/','-')#####消除文件名中'/'
    bookinfo.append(Gclass)
    bookinfo.append(sclassion)
    #print(bookinfo)
    # SQL 插入语句
    sql = "INSERT INTO DANGDANGBOOK(booktitle,publicationtime,author,press,bookprice,bookdetail,Gclass,sclassion )\
       VALUES ('%s','%s','%s','%s','%f','%s','%s','%s')" % \
       (bookinfo[0],bookinfo[1],bookinfo[2],bookinfo[3],bookinfo[4],bookinfo[5],bookinfo[6],bookinfo[7])
    try:
       # 执行sql语句
       cursor.execute(sql)
       # 提交到数据库执行
       db.commit()
    except:
       # 如果发生错误则回滚
       db.rollback()
       print('erron sql----{0[0]},{0[6]},{0[7]}'.format(bookinfo))

In [ ]:
if __name__=='__main__':
    # 打开数据库连接
    db = pymysql.connect(host="62.234.149.37",user="root",passwd="smile",db="testdb" ,charset='utf8')#########一定要重启mysql
    ####(host='127.0.0.1',user='root',passwd='root',db='pmdb',charset='utf8')
    # 使用 cursor() 方法创建一个游标对象 cursor
    cursor = db.cursor()
    # 使用 execute() 方法执行 SQL，如果表存在则删除
    cursor.execute("DROP TABLE IF EXISTS DANGDANGBOOK")
    # 使用预处理语句创建表
    sql = """CREATE TABLE DANGDANGBOOK(
           booktitle VARCHAR(10) ,
           publicationtime VARCHAR(10) ,
           author VARCHAR(10),
           press VARCHAR(15), 
           bookprice FLOAT,
           bookdetail VARCHAR(20),
           Gclass VARCHAR(5),
           sclassion VARCHAR(5)
        )ENGINE=InnoDB  DEFAULT CHARSET=utf8"""
    ##PRIMARY KEY ( `booktitle` )
    cursor.execute(sql)
    print('表创建完成')
    ##############表创建完成
    
    Gclass,sclassion,sclassurl,lensclass=getClassification()#####50 679 679 50  ######length
    getbookinfo(Gclass,sclassion,sclassurl,lensclass)


    # 关闭数据库连接
    db.close()

表创建完成
erron sql----女不强大天不容,青春文学,影视写真
erron sql----大汉情缘之云中歌纪念,青春文学,影视写真
erron sql----【随书赠送10张主演,青春文学,影视写真
erron sql----《你的名字。》官方视,青春文学,穿越-重生-架空
erron sql----山河表里 上卷/下卷,青春文学,穿越-重生-架空
erron sql----纨绔世子妃完美套装（,青春文学,穿越-重生-架空
erron sql----梦溪石：麟趾（套装全,青春文学,穿越-重生-架空
erron sql----看清楚再下单】侯卫东,青春文学,穿越-重生-架空
erron sql----法老的宠妃1·时空黄,青春文学,穿越-重生-架空
erron sql----法老的宠妃2·荷鲁斯,青春文学,穿越-重生-架空
erron sql----夜旅人,青春文学,穿越-重生-架空
erron sql----永夜君王 卷一：在永,青春文学,穿越-重生-架空
erron sql----凤逆苍穹（1-4）套,青春文学,穿越-重生-架空
erron sql----再生缘：我的温柔暴君,青春文学,穿越-重生-架空
erron sql----《莲花签》（套装共3,青春文学,穿越-重生-架空
erron sql----《错嫁良缘4海盗千金,青春文学,穿越-重生-架空
erron sql----大宋的智慧1,青春文学,穿越-重生-架空
erron sql----恨倾城,青春文学,穿越-重生-架空
erron sql----鸭川六景（奇幻京都风,青春文学,穿越-重生-架空
erron sql----十一倦（套装全二册）,青春文学,穿越-重生-架空
erron sql----承君眷顾,青春文学,穿越-重生-架空
erron sql----《寻找前世之旅续集》,青春文学,穿越-重生-架空
erron sql----现货 法老的宠妃1 ,青春文学,穿越-重生-架空
erron sql----大宋的智慧3,青春文学,穿越-重生-架空
erron sql----错嫁良缘1-6六部曲,青春文学,穿越-重生-架空
erron sql----别拿穿越不当工作（全,青春文学,穿越-重生-架空
erron sql----大宋的智慧2,青春文学,穿越-重生-架空
erron sql---- 悦

erron sql----斗罗大陆（精装典藏版,青春文学,玄幻-新武侠-魔幻-科幻
erron sql----斗罗大陆3龙王传说1,青春文学,玄幻-新武侠-魔幻-科幻
erron sql----斗罗大陆2 绝世唐门,青春文学,玄幻-新武侠-魔幻-科幻
erron sql----斗破苍穹全集第二辑 ,青春文学,玄幻-新武侠-魔幻-科幻
erron sql----元尊6·逆流而上,青春文学,玄幻-新武侠-魔幻-科幻
erron sql----元尊9·玄源洞天,青春文学,玄幻-新武侠-魔幻-科幻
erron sql----爵迹套装2018版（,青春文学,玄幻-新武侠-魔幻-科幻
erron sql----我城,青春文学,港台青春文学
erron sql----鲸鱼女孩 池塘男孩（,青春文学,港台青春文学
erron sql----一百个我想你（精装 ,青春文学,港台青春文学
erron sql----不怕青春太疼痛，只怕,青春文学,港台青春文学
erron sql----衣衣不舍-台湾纯爱言,青春文学,港台青春文学
erron sql----武道狂之诗·卷八 破,青春文学,港台青春文学
erron sql----藤井树真情三部曲 六,青春文学,港台青春文学
erron sql----我想要的只是一个拥抱,青春文学,港台青春文学
erron sql----蔡智恒文集 懈寄生,青春文学,港台青春文学
erron sql----上课不要看小说(“上,青春文学,港台青春文学
erron sql----交错时光的爱恋（古典,青春文学,港台青春文学
erron sql----卖时间的商店,青春文学,港台青春文学
erron sql----等一个人咖啡,青春文学,港台青春文学
erron sql----巴黎圣母院 雨果原著,青春文学,港台青春文学
erron sql----【送有声书】巴黎圣母,青春文学,港台青春文学
erron sql----猜猜我有多爱你（两届,青春文学,港台青春文学
erron sql----边荒传说（卷十四）—,青春文学,港台青春文学
erron sql----边荒传说（卷十一）—,青春文学,港台青春文学
erron sql----最美的，徒劳无功(九,青春文学,港台青春文学
erron sql----第一次的亲密接触,青春文学,港台青春文学
erron sql

erron sql----金庸作品集(朗声旧版,小说,中国当代小说
erron sql----尘埃落定,小说,中国当代小说
erron sql----应物兄,小说,中国当代小说
erron sql----六里庄遗事,小说,中国当代小说
erron sql----清单人生（霍思燕推荐,小说,中国当代小说
erron sql----灿烂千阳,小说,中国当代小说
erron sql----巨流河（平装本）,小说,中国当代小说
erron sql----黄金时代（手稿珍藏版,小说,中国当代小说
erron sql----冷场（李诞2018年,小说,中国当代小说
erron sql----仇敌1-5,小说,都市
erron sql----全中国最穷的小伙子发,小说,都市
erron sql----小王子（畅销300万,小说,世界名著
erron sql----克苏鲁神话合集,小说,魔幻
erron sql----肠子（无删节版）,小说,惊悚-恐怖
erron sql----东野圭吾：恶意（20,小说,侦探-悬疑-推理
erron sql----东野圭吾：嫌疑人X的,小说,侦探-悬疑-推理
erron sql----放学后（东野圭吾成名,小说,侦探-悬疑-推理
erron sql----东野圭吾四大推理套装,小说,侦探-悬疑-推理
erron sql----东野圭吾：悲剧人偶,小说,侦探-悬疑-推理
erron sql----祈祷落幕时,小说,侦探-悬疑-推理
erron sql----无人生还：阿加莎 克,小说,侦探-悬疑-推理
erron sql----东野圭吾：时生（东野,小说,侦探-悬疑-推理
erron sql----新参者（2016版）,小说,侦探-悬疑-推理
erron sql----东野圭吾：禁断的魔术,小说,侦探-悬疑-推理
erron sql----东野圭吾：侦探伽利略,小说,侦探-悬疑-推理
erron sql----肖申克的救赎 （修订,小说,侦探-悬疑-推理
erron sql----黄金降落伞（风靡欧美,小说,侦探-悬疑-推理
erron sql----红手指（2016版）,小说,侦探-悬疑-推理
erron sql----鬼吹灯全集（ 套装共,小说,侦探-悬疑-推理
erron sql----东野圭吾：假面山庄（,小说,侦探-悬疑-推理
erron

erron sql----古典诗歌的生命情怀,文学,文学评论与鉴赏
erron sql----晚唐（精装）,文学,文学评论与鉴赏
erron sql----木心谈木心：《文学回,文学,文学评论与鉴赏
erron sql----最糟的宇宙，最好的地,文学,文学评论与鉴赏
erron sql----小于一,文学,文学评论与鉴赏
erron sql----草木缘情：中国古典文,文学,文学评论与鉴赏
erron sql----既见君子——过去时代,文学,文学评论与鉴赏
erron sql----经典名著的人生智慧,文学,文学评论与鉴赏
erron sql----马瑞芳品读红楼梦(上,文学,文学评论与鉴赏
erron sql----重读八十年代,文学,文学评论与鉴赏
erron sql----史记选（中国古典文学,文学,文学评论与鉴赏
erron sql----蒋勋说文学之美（五册,文学,文学评论与鉴赏
erron sql----中国文学鉴赏辞典（新,文学,文学评论与鉴赏
erron sql----经典小说解读(魅力经,文学,文学评论与鉴赏
erron sql----苏轼诗文鉴赏辞典,文学,文学评论与鉴赏
erron sql----文化与帝国主义（二版,文学,文学评论与鉴赏
erron sql----纳博科夫最喜欢的词,文学,文学评论与鉴赏
erron sql----温和地走进宋词的凉夜,文学,文学评论与鉴赏
erron sql----小王子的领悟,文学,文学评论与鉴赏
erron sql----从卡夫卡到昆德拉：2,文学,文学评论与鉴赏
erron sql----孙绍振解读经典散文,文学,文学评论与鉴赏
erron sql----永生的操练——但丁《,文学,文学评论与鉴赏
erron sql----浮生六记（汪涵、胡歌,文学,中国古代随笔
erron sql----浮生六记（详细注解+,文学,中国古代随笔
erron sql----年轻人要熟知的424,文学,中国古代随笔
erron sql----浮生六记（珍藏本）,文学,中国古代随笔
erron sql----浮生六记（全新修订版,文学,中国古代随笔
erron sql----浮生六记 先锋经典文,文学,中国古代随笔
erron sql----浮生六记{外三种}（,文学,中国古代随笔
erron sql----陶庵梦忆 

erron sql----中国人的生活美学：浮,文学,中国现当代诗歌
erron sql----陪孩子读最美古诗词（,文学,中国现当代诗歌
erron sql----鲁迅文集：野草,文学,中国现当代诗歌
erron sql----北岛集  在天涯：诗,文学,中国现当代诗歌
erron sql----北岛集  城门开,文学,中国现当代诗歌
erron sql----顾城的诗顾城的画（精,文学,中国现当代诗歌
erron sql----北岛集  履历：诗选,文学,中国现当代诗歌
erron sql----汪国真诗精编：名家经,文学,中国现当代诗歌
erron sql----微笑总在装饰我的梦—,文学,中国现当代诗歌
erron sql----穆旦诗集,文学,中国现当代诗歌
erron sql----朦胧诗精编：名家经典,文学,中国现当代诗歌
erron sql----海子诗选权威珍藏本,文学,中国现当代诗歌
erron sql----在最深的红尘里重逢（,文学,中国现当代诗歌
erron sql----海子经典诗全集（全二,文学,中国现当代诗歌
erron sql----你来人间一趟，你要看,文学,中国现当代诗歌
erron sql----汪国真诗精选,文学,中国现当代诗歌
erron sql----西班牙三棵树（精装）,文学,中国现当代诗歌
erron sql----毛泽东诗词大观,文学,中国现当代诗歌
erron sql----海桑诗集全三册 读库,文学,中国现当代诗歌
erron sql----顾城诗精编：名家经典,文学,中国现当代诗歌
erron sql----关于方文山的素颜韵脚,文学,中国现当代诗歌
erron sql----摇摇晃晃的人间（余秀,文学,中国现当代诗歌
erron sql----十月怀胎知识百科全书,孕产-胎教,孕期
erron sql----家庭教育书籍全套3册,孕产-胎教,英文原版书-孕产胎教
erron sql----全4册 孕妈妈护理专,孕产-胎教,英文原版书-孕产胎教
erron sql----Belly Laug,孕产-胎教,英文原版书-孕产胎教
erron sql----晚安双语胎教故事朗读,孕产-胎教,英文原版书-孕产胎教
erron sql----YOU:孕婴使用手册,孕产-胎教,孕产妇健康
erron sql----范志红

...

erron sql----青春期的烦恼——女生,亲子-家教,13-18岁
erron sql----换种方式说，青春期孩,亲子-家教,13-18岁
erron sql----青春期的烦恼——男生,亲子-家教,13-18岁
erron sql----尹建莉的书作品正版 ,亲子-家教,品格养成
erron sql----婚恋与两性套装5册 ,两性关系,性
erron sql----性爱的艺术,两性关系,性
erron sql----爱的五种语言：创造完,两性关系,性
erron sql----渡边淳一经典作品集3,两性关系,婚姻
erron sql----婚姻家庭咨询师（国家,两性关系,婚姻
erron sql----爱的五种语言+道歉的,两性关系,两性关系
erron sql----新生儿婴儿护理百科全,育儿-早教,育儿百科
erron sql----幼儿版 我爱乐理1 ,育儿-早教,英文原版书-育儿早教
erron sql----The Little,育儿-早教,英文原版书-育儿早教
erron sql----妈妈陪你考清华：兔妈,育儿-早教,英文原版书-育儿早教
erron sql----内向性格的竞争力（青,育儿-早教,英文原版书-育儿早教
erron sql----10 Things ,育儿-早教,英文原版书-育儿早教
erron sql----学校图书馆推荐Rum,育儿-早教,英文原版书-育儿早教
erron sql----外国少年儿童情趣钢琴,育儿-早教,英文原版书-育儿早教
erron sql----All Aboard,育儿-早教,英文原版书-育儿早教
erron sql----英文原版绘本 Fir,育儿-早教,英文原版书-育儿早教
erron sql----正版现货 蝴蝶狮 英,育儿-早教,英文原版书-育儿早教
erron sql----欧美宝宝必读绘本4本,育儿-早教,英文原版书-育儿早教
erron sql----世界上最聪明的孩子 ,育儿-早教,英文原版书-育儿早教
erron sql----I can read,育儿-早教,英文原版书-育儿早教
erron sql----反溺爱（在日趋势物质,育儿-早教,英文原版书-育儿早教
erron sql----正版虎妈战歌 英文原,育儿-早教,英文原版书-育儿早教
erron sql----孩子 挑战

erron sql----宝宝免疫力这么调,育儿-早教,婴幼儿护理健康
erron sql----医生的叮咛——早产宝,育儿-早教,婴幼儿护理健康
erron sql----给宝宝一口好牙齿——,育儿-早教,婴幼儿护理健康
erron sql----婴幼儿护理与习惯养成,育儿-早教,婴幼儿护理健康
erron sql----新生儿婴儿护理大百科,育儿-早教,婴幼儿护理健康
erron sql----新生儿婴儿护理百科全,育儿-早教,婴幼儿护理健康
erron sql----《0~3岁婴幼儿护理,育儿-早教,婴幼儿护理健康
erron sql----崔玉涛套装四册,育儿-早教,婴幼儿护理健康
erron sql----双胞胎睡眠圣经,育儿-早教,婴幼儿护理健康
erron sql----只有儿科医生知道（凤,育儿-早教,婴幼儿护理健康
erron sql----DK宝宝健康与疾病百,育儿-早教,婴幼儿护理健康
erron sql----居家养老照料,保健-养生,中老年养生
erron sql----全民健康膳食宝典（汉,保健-养生,药膳食疗
erron sql----广东清润糖水,保健-养生,药膳食疗
erron sql----跑步圣经 （第2版）,保健-养生,运动健康
erron sql----健身营养全书——关于,保健-养生,运动健康
erron sql----力量训练计划：用精准,保健-养生,运动健康
erron sql----艾扬格少儿瑜伽(基础,保健-养生,运动健康
erron sql----艾扬格少儿瑜伽（精进,保健-养生,运动健康
erron sql----拉伸训练彩色图谱,保健-养生,运动健康
erron sql----华西医院辟谣小分队医,保健-养生,饮食健康
erron sql----华西医院辟谣小分队医,保健-养生,饮食健康
erron sql----中国妇幼人群膳食指南,保健-养生,饮食健康
erron sql----华西医院辟谣小分队医,保健-养生,饮食健康
erron sql----本草纲目 李时珍 原,保健-养生,英文原版书-保健养生
erron sql----Testostero,保健-养生,英文原版书-保健养生
erron sql----Chemothera,保健-养生,英文原版书-保健养生
erron sql----Dsm-Iv-T

erron sql----人类动作发展概论,体育-运动,体育理论与教学
erron sql----怀斯曼体能训练手册,体育-运动,体育理论与教学
erron sql----HIIT 高强度间歇,体育-运动,体育理论与教学
erron sql----体育简史,体育-运动,体育理论与教学
erron sql----刷新PB：跑步提速指,体育-运动,体育理论与教学
erron sql----运动生理学 人民体育,体育-运动,体育理论与教学
erron sql----运动生物力学,体育-运动,体育理论与教学
erron sql----中国传统武术史,体育-运动,体育理论与教学
erron sql----核心力量体能训练法,体育-运动,体育理论与教学
erron sql----间歇训练全书 练就超,体育-运动,体育理论与教学
erron sql----跑者脑力训练手册,体育-运动,体育理论与教学
erron sql----羽毛球技术进阶训练 ,体育-运动,体育理论与教学
erron sql----明年更年轻:运动赋能,体育-运动,体育理论与教学
erron sql----从优秀到卓越 如何做,体育-运动,体育理论与教学
erron sql----体育教学理论问题与案,体育-运动,体育理论与教学
erron sql----青少年体育运动指导与,体育-运动,体育理论与教学
erron sql----走路瘦肚,体育-运动,体育理论与教学
erron sql----力量突破 重塑肌肉形,体育-运动,体育理论与教学
erron sql----体育活动策划与管理,体育-运动,体育理论与教学
erron sql----体育保健与运动康复技,体育-运动,体育理论与教学
erron sql----动作学习与控制,体育-运动,体育理论与教学
erron sql----【抖音同款书】肌肉力,体育-运动,体育理论与教学
erron sql----自由风格训练 4个基,体育-运动,体育理论与教学
erron sql----运动康复消疼痛：不开,体育-运动,体育理论与教学
erron sql----形体训练与形象设计,体育-运动,体育理论与教学
erron sql----美国创新领导力中心（,体育-运动,体育理论与教学
erron sql----中国足球产业与文化发,体育-运动,体育理论与教学
erro

erron sql----把刺绣变简单：100,手工-DIY,刺绣-十字绣
erron sql----刺绣经典针法与图案.,手工-DIY,刺绣-十字绣
erron sql----韩风花草绣：花、树、,手工-DIY,刺绣-十字绣
erron sql----刺子绣图典,手工-DIY,刺绣-十字绣
erron sql----最详尽的刺绣教科书,手工-DIY,刺绣-十字绣
erron sql----青木和子的刺绣旅行手,手工-DIY,刺绣-十字绣
erron sql----言叶之绣,手工-DIY,刺绣-十字绣
erron sql----300例植物主题刺绣,手工-DIY,刺绣-十字绣
erron sql----刺绣经典针法与图案.,手工-DIY,刺绣-十字绣
erron sql----Cotton tim,手工-DIY,刺绣-十字绣
erron sql----户冢贞子的绝美刺绣.,手工-DIY,刺绣-十字绣
erron sql----北村绘里的小小刺绣,手工-DIY,刺绣-十字绣
erron sql----草乃静的如梦似幻日本,手工-DIY,刺绣-十字绣
erron sql----毛线球.21，拉脱维,手工-DIY,刺绣-十字绣
erron sql----清新花草刺绣580款,手工-DIY,刺绣-十字绣
erron sql----尾上惠美刺绣针法图典,手工-DIY,刺绣-十字绣
erron sql----户冢贞子的绝美刺绣：,手工-DIY,刺绣-十字绣
erron sql----樋口愉美子的12个月,手工-DIY,刺绣-十字绣
erron sql----大塚绚子的优美立体刺,手工-DIY,刺绣-十字绣
erron sql----户冢贞子的绝美刺绣.,手工-DIY,刺绣-十字绣
erron sql----刺绣线钩织的绚丽发饰,手工-DIY,刺绣-十字绣
erron sql----樋口愉美子的贴布缝刺,手工-DIY,刺绣-十字绣
erron sql----爱刺绣6：刺绣物语,手工-DIY,刺绣-十字绣
erron sql----爱刺绣5，追忆往事的,手工-DIY,刺绣-十字绣
erron sql----樋口愉美子的双色刺绣,手工-DIY,刺绣-十字绣
erron sql----青木和子的刺绣日记：,手工-DIY,刺绣-十字绣
erron sql----优美典雅的细工花饰,手工-

erron sql----静香的花道生活:日式,手工-DIY,其他
erron sql----【抖音同款赠相框+刻,手工-DIY,其他
erron sql----手编大好！6：20周,手工-DIY,其他
erron sql----手缝皮革技巧事典,手工-DIY,其他
erron sql----永生花设计与制作,手工-DIY,其他
erron sql----绣线钩编的植物小装饰,手工-DIY,其他
erron sql----迷上曼陀罗钩编——花,手工-DIY,其他
erron sql----木工雕刻全书,手工-DIY,其他
erron sql----草木染大全,手工-DIY,其他
erron sql----用UV胶制作压花小饰,手工-DIY,其他
erron sql----爸爸的木匠小屋：创意,手工-DIY,其他
erron sql----最基础的10种树形盆,手工-DIY,其他
erron sql----文具手帖：偶尔相见特,手工-DIY,其他
erron sql----零基础家庭小木工,手工-DIY,其他
erron sql----陶土手作基础教科书：,手工-DIY,其他
erron sql----幸福手作 超轻黏土—,手工-DIY,其他
erron sql----自己做100%保养级,手工-DIY,其他
erron sql----cotton tim,手工-DIY,其他
erron sql----我们一起画石头吧：在,手工-DIY,其他
erron sql----齐藤谣子的拼布教室（,手工-DIY,其他
erron sql----木工大师教你做36款,手工-DIY,其他
erron sql----柳川昌子压花技法,手工-DIY,其他
erron sql----无师自通学手工：皮革,手工-DIY,其他
erron sql----洋裁大百科,手工-DIY,其他
erron sql----幸福手作 超轻黏土—,手工-DIY,其他
erron sql----文具手帖：偶尔相见特,手工-DIY,其他
erron sql----UV胶饰品制作入门大,手工-DIY,其他
erron sql----小日子：欠削吧，木头,手工-DIY,其他
erron sql----娜娜阿库娅的魅力热缩,手工-DIY,其他
erron sql----女孩日常手作服的裁剪,手工-DIY,其他
er